## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header = True, inferSchema = True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.select('time').show()

+------+
|  time|
+------+
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
|Dinner|
+------+
only showing top 20 rows



In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[6]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
###Handling categorical features in Pyspark
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols = ['sex', 'smoker', 'day', 'time'], outputCols = ['sex_i', 'smoker_i', 'day_i', 'time_i'])

In [0]:
df_1 = indexer.fit(df).transform(df)
df_1.show()

+----------+----+------+------+---+------+----+-----+--------+-----+------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_i|smoker_i|day_i|time_i|
+----------+----+------+------+---+------+----+-----+--------+-----+------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|  1.0|     0.0|  1.0|   0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|  1.0|     0.0|  1.0|   0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|
|     10.27|

In [0]:
###creating independent feature
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['tip', 'size', 'sex_i', 'smoker_i', 'day_i', 'time_i'], outputCol = 'IndependentFeature')
output = featureassembler.transform(df_1)

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----+--------+-----+------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_i|smoker_i|day_i|time_i|  IndependentFeature|
+----------+----+------+------+---+------+----+-----+--------+-----+------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|  1.0|     0.0|  1.0|   0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  0.0|     0.0|  1.0|   0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|  1.0|     0.0|  1.0|   0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|  0.0|     0.0|  1.0|   0.0|[4.71,4.0,0.0,0.0...|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  0.0|     0.0|  1.0|   0.0|[2.0,2.0,0.0,0.0,...|
|     26.88|3.12|  Male|    No

In [0]:
finalized_data = output.select('IndependentFeature', 'total_bill')
finalized_data.show()

+--------------------+----------+
|  IndependentFeature|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
#Regression model
from pyspark.ml.regression import LinearRegression
train, test = finalized_data.randomSplit([.75, .25])
regressor = LinearRegression(featuresCol = 'IndependentFeature', labelCol = 'total_bill')
regressor = regressor.fit(train)

In [0]:
regressor.coefficients

Out[28]: DenseVector([3.1131, 3.4363, -1.2776, 2.1484, -0.3988, -0.9909])

In [0]:
regressor.intercept

Out[30]: 1.8286691536377238

In [0]:
prediction = regressor.evaluate(test)

In [0]:
prediction.predictions.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+--------------------+----------+------------------+
|  IndependentFeature|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.51|12.592710990100256|
|(6,[0,1],[1.75,2.0])|     17.82|14.149276855979261|
| (6,[0,1],[2.0,2.0])|     13.37|14.927559788918764|
| (6,[0,1],[2.0,3.0])|     16.31|18.363873374801273|
|(6,[0,1],[2.31,3.0])|     18.69|19.328944211646256|
|(6,[0,1],[2.64,3.0])|     17.59|20.356277683126397|
|(6,[0,1],[3.15,3.0])|     20.08|21.943974866322986|
| (6,[0,1],[4.3,2.0])|      21.7|22.087762771962186|
|(6,[0,1],[5.92,3.0])|     29.03|30.567349763292672|
|[1.44,2.0,0.0,1.0...|      7.74| 15.33265407312154|
|[1.5,2.0,0.0,1.0,...|     11.59| 15.51944197702702|
|[1.5,2.0,1.0,0.0,...|     10.65|10.304792219780017|
|[1.5,2.0,1.0,0.0,...|     11.17|10.304792219780017

In [0]:
prediction.r2, prediction.meanAbsoluteError, prediction.meanSquaredError

Out[37]: (0.4619173857107136, 3.531199432563328, 32.601606836216654)